The first aim of this notebook is to illustrate the scientist's problem when fitting a decision making model. 

To start simple, the decision space is binary and the stimulus space is the continuum $[0,1]$.

I propose to simulate data from a decision maker unknown to the public, let's call it $D_h$.
Then, I propose to offer two candidate models $D_1, D_2$ to explain the data. Without the public knowing, I set $D_1=D_h$ and $D_2\neq D_h$. I would like to design $D_1$ and $D_2$ carefully so that I can trump the public in believing that $D_2$ surpasses $D_1$ in regards to its 'explanatory power'.

In [69]:
from scipy.stats import bernoulli, uniform
import numpy as np

In [70]:
class BernoulliDecisionMaker:
    """
    This decision maker is oblivious to the stimulus, it is a pure Bernoulli r.v.
    """
    def __init__(self, name, p=None, bias=0):
        """
        :param p: sucess probability of r.v.
        """
        self.name = name
        self.p = p
        self.bias = bias
        self.decisions = None
        self.perceived_stimulus = None
        self.accuracy = None
        
    def __str__(self):
        return f"name: {self.name}\n" \
               f"bias: {self.bias}\n" \
               f"accuracy: {self.accuracy}\n"
                
    def compute_accuracy(self, correct):
        if self.decisions is None:
            f"No decisions have been made"
        else:
            self.accuracy = np.sum(self.decisions == correct) / len(correct)
    
    def decide(self, n):
        """
        makes n decisions, independent of stimulus, and using constant success probability
        :param n: number of independent samples to draw
        """
        self.decisions = bernoulli.rvs(self.p, size=n)
    
    def present_stimulus(self, s):
        """
        modifies stimulus s as if its perception were biased towards the extremes {0,1}
        :param s: array of stimulus values between 0 and 1
        :param bias: number between 0 and 1
        """
        if self.bias == 0:
            self.perceived_stimulus = s
        else:
            def apply_bias(ss):
                gap = (1 - self.bias) / 2
                if ss < 0.5:
                    return uniform.rvs(loc=0,scale=gap,size=1)
                else:
                    return uniform.rvs(loc=0.5 + self.bias/2, scale=gap, size=1)
            self.perceived_stimulus = np.array([apply_bias(sss) for sss in s])
    
    @classmethod
    def bernoulli_decide(cls, probabilities):
        """
        makes N Bernoulli decisions with specified success probabilities, where N is the length of 'probabilities'
        :param probabilities: array of success probabilities
        """
        return np.array([bernoulli.rvs(pp) for pp in probabilities])

In [71]:
class StimulusUnitInterval:
    sequence = None
    
    def __init__(self, size=10):
        self.size = size
        self.sequence = None
        self.correct = None
    
    def __str__(self):
        return f"stim 1-10: {str([round(x,2) for x in self.sequence[0:10]])}\n" \
               f"correct 1-10: {str(self.correct[0:10])}"
    
    def generate_sequence(self, mode):
        if mode == 'random':
            self.sequence = self.generate_random_sequence(self.size)
        else: 
            raise ValueError(f"only supported mode is 'random' for now")
        self.correct = self.sequence > 0.5
        
    @classmethod
    def generate_random_sequence(cls, n):
        return uniform.rvs(size=n)

In [72]:
stim = StimulusUnitInterval()
stim.size = 1000
stim.generate_sequence('random')
print(stim)

stim 1-10: [0.33, 0.95, 0.53, 0.48, 0.28, 0.82, 0.31, 0.97, 0.22, 0.02]
correct 1-10: [False  True  True False False  True False  True False False]


In [73]:
# decision makers
Dh = BernoulliDecisionMaker('Dh')
D1 = BernoulliDecisionMaker('D1')
D2 = BernoulliDecisionMaker('D2', bias=.7)

decision_makers = {Dh, D1, D2}

# generate decisions with all of them
for D in decision_makers:
    D.present_stimulus(stim.sequence)
    D.decisions = D.bernoulli_decide(D.perceived_stimulus)
    D.compute_accuracy(stim.correct)

In [74]:
# explore decision makers and compute some statistics
for D in decision_makers:
    print(D)
#     print(stim)
#     print(D.perceived_stimulus)
#     print(D.decisions)
#     print('\n')

name: D2
bias: 0.7
accuracy: 0.926

name: Dh
bias: 0
accuracy: 0.735

name: D1
bias: 0
accuracy: 0.741



In [75]:
# compare models in terms of percent match
for D in decision_makers - {Dh}:
    D.percent_match = np.sum(D.decisions == Dh.decisions) / len(stim.sequence)
    print(f"{D.name} {round(100 * D.percent_match,2)}%")

D2 70.9%
D1 66.8%
